## This script loads the current model and performs an evaluation of it

### Initialize
First, initialize the model with all parameters


In [ ]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
from model import Model
from model_relu_old import ModelOld
from loss import TripletLoss, ImprovedTripletLoss
from training_set import TrainingSet
from average_meter import AverageMeter
from data_splitter import DataSplitter
from mission_indices import MissionIndices
from database_parser import DatabaseParser

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import pyshtools
from pyshtools import spectralanalysis
from pyshtools import shio
from pyshtools import expand

import sys
import time
import math
import operator
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from tqdm.auto import tqdm
import scipy.stats as st
from scipy import spatial

%reload_ext autoreload
%autoreload 2

In [ ]:
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True
n_features = 3
bandwidth = 100
from model_relu_old import ModelOld
net = Model(n_features, bandwidth).cuda()
#net = ModelOld(n_features, bandwidth).cuda()
restore = False
optimizer = torch.optim.SGD(net.parameters(), lr=5e-3, momentum=0.9)
batch_size = 12
num_workers = 12
descriptor_size = 256
net_input_size = 2*bandwidth
cache = 50
criterion = ImprovedTripletLoss(margin=2, alpha=0.5, margin2=0.2)
writer = SummaryWriter()
stored_model = './net_params_arche_high_res_big.pkl'
net.load_state_dict(torch.load(stored_model))
summary(net, input_size=[(3, 200, 200), (3, 200, 200), (3, 200, 200)])

Initialize the data source

In [ ]:
#dataset_path = "/media/scratch/berlukas/spherical/arche_high_res2/"
dataset_path = "/home/berlukas/data/arche_low_res2/"
db_parser = DatabaseParser(dataset_path)

training_missions, test_missions = MissionIndices.get_arche_low_res()
training_indices, test_indices = db_parser.extract_training_and_test_indices(
    training_missions, test_missions)
print(f'Found {len(test_missions)} test indices.')

n_test_data = 10500
n_test_cache = n_test_data
ds_test = DataSource(dataset_path, n_test_cache, -1, False)
idx = np.array(test_indices['idx'].tolist())
ds_test.load(n_test_data, idx, filter_clusters=True)
n_test_data = len(ds_test.anchors)

In [ ]:
test_set = TrainingSet(restore, bandwidth)
test_set.generateAll(ds_test)

In [ ]:
# hack for removing the images
#test_set.anchor_features = test_set.anchor_features[:,0:2,:,:]
#test_set.positive_features = test_set.positive_features[:,0:2,:,:]
#test_set.negative_features = test_set.negative_features[:,0:2,:,:]


n_test_set = len(test_set)
print("Total size: ", n_test_set)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1, pin_memory=True, drop_last=False)

  ## Generate the descriptors for anchor and positive

In [ ]:
def accuracy(dista, distb):
    margin = 0
    pred = (dista - distb - margin).cpu().data
    acc = ((pred < 0).sum()).float()/dista.size(0)
    return acc

net.eval()
n_iter = 0
anchor_embeddings = np.empty(1)
positive_embeddings = np.empty(1)
with torch.no_grad():
    test_accs = AverageMeter()
    test_pos_dist = AverageMeter()
    test_neg_dist = AverageMeter()
    start = time.time()
    for batch_idx, (data1, data2) in enumerate(test_loader):
        embedded_a, embedded_p, embedded_n = net(data1.cuda().float(), data2.cuda().float(), data2.cuda().float())
        dist_to_pos, dist_to_neg, loss, loss_total = criterion(embedded_a, embedded_p, embedded_n)
        #writer.add_scalar('Ext_Test/Loss', loss, n_iter)

        acc = accuracy(dist_to_pos, dist_to_neg)
        test_accs.update(acc, data1.size(0))
        test_pos_dist.update(dist_to_pos.cpu().data.numpy().sum())
        test_neg_dist.update(dist_to_neg.cpu().data.numpy().sum())

        #writer.add_scalar('Ext_Test/Accuracy', test_accs.avg, n_iter)
        #writer.add_scalar('Ext_Test/Distance/Positive', test_pos_dist.avg, n_iter)
        #writer.add_scalar('Ext_Test/Distance/Negative', test_neg_dist.avg, n_iter)

        anchor_embeddings = np.append(anchor_embeddings, embedded_a.cpu().data.numpy().reshape([1,-1]))
        positive_embeddings = np.append(positive_embeddings, embedded_p.cpu().data.numpy().reshape([1,-1]))
        n_iter = n_iter + 1
    end = time.time()
    print(f'Duration per batch {(end - start)/n_test_set}s')    
desc_anchors = anchor_embeddings[1:].reshape([n_test_set, descriptor_size])
desc_positives = positive_embeddings[1:].reshape([n_test_set, descriptor_size])      

In [ ]:
0.1377246098987184/3

## Simple old testing pipeline (index based)

In [ ]:
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 0.05
max_anchor_dist = 1
for n_nearest_neighbors in tqdm(range(1,21)):
    pos_count = 0
    anchor_count = 0
    idx_count = 0
    for idx in range(n_test_set):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(desc_positives[nn_i,:], desc_positives[idx,:])
            if (dist <= max_pos_dist):
                pos_count = pos_count + 1;
                break
        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(desc_positives[nn_i,:], desc_anchors[idx,:])
            if (dist <= max_anchor_dist):
                anchor_count = anchor_count + 1;
                break
        for nn_i in nn_indices:
            if (nn_i == idx):
                idx_count = idx_count + 1;
                break
    pos_precision = (pos_count*1.0) / n_test_set
    anchor_precision = (anchor_count*1.0) / n_test_set
    idx_precision = (idx_count*1.0) / n_test_set
    
    print(f'recall {idx_precision} for {n_nearest_neighbors} neighbors')
    #writer.add_scalar('Ext_Test/Precision/Positive_Distance', pos_precision, n_nearest_neighbors)
    #writer.add_scalar('Ext_Test/Precision/Anchor_Distance', anchor_precision, n_nearest_neighbors)
    #writer.add_scalar('Ext_Test/Precision/Index_Count', idx_precision, n_nearest_neighbors)

## New testing pipeline (location based)

In [ ]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0
max_anchor_dist = 1
anchor_poses = ds_test.anchor_poses
positive_poses = ds_test.positive_poses
assert len(anchor_poses) == len(positive_poses)

for n_nearest_neighbors in tqdm(range(1,21)):    
    loc_count = 0
    for idx in range(0, n_test_set):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                loc_count = loc_count + 1;
                break
                
    loc_precision = (loc_count*1.0) / n_test_set    
    #print(f'recall {loc_precision} for {n_nearest_neighbors} neighbors')
    print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/Location', loc_precision, n_nearest_neighbors)    

## Place Voting using Global Spectral Analysis


In [ ]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
start = time.time()
tree = spatial.KDTree(desc_positives)
end = time.time()
print(f'Duration for building the kd-tree {(end - start)}s')   
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds_test.anchor_poses
anchor_clouds = ds_test.anchors
anchor_features = test_set.anchor_features

positive_poses = ds_test.positive_poses
positive_clouds = ds_test.positives
positive_features = test_set.anchor_features

for n_nearest_neighbors in tqdm(range(1,2)):        
    n_matches = 0
    loc_count = 0    
    dur_neighbor_processing_s = 0
    for idx in range(0, n_test_set):                
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        
        z_scores = [0] * n_nearest_neighbors
        contains_match = False        
        true_match_idx = 0
        start = time.time()
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_test_set):
                print(f'ERROR: index {nn_i} is outside of {n_data}')
                break;
                
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True                
                true_match_idx = i
                
            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
            a_intensity = anchor_features[idx][1,:,:]
            p_intensity = positive_features[nn_i][1,:,:]
            #a_img = anchor_features[idx][2,:,:]
            #p_img = positive_features[nn_i][2,:,:]
                                               
            a_range_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_range_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)
            
            a_intensity_coeffs = pyshtools.expand.SHExpandDH(a_intensity, sampling=1)
            p_intensity_coeffs = pyshtools.expand.SHExpandDH(p_intensity, sampling=1)
            
            #a_img_coeffs = pyshtools.expand.SHExpandDH(a_img, sampling=1)
            #p_img_coeffs = pyshtools.expand.SHExpandDH(p_img, sampling=1)
            
            #a_fused = np.empty([3, a_range_coeffs.shape[0], a_range_coeffs.shape[1]])
            #p_fused = np.empty([3, p_range_coeffs.shape[0], p_range_coeffs.shape[1]])
            #print(a_range_coeffs.shape)
            #a_fused[0,:] = a_range_coeffs
            
            
            admit, error, corr = spectralanalysis.SHAdmitCorr(a_range_coeffs, p_range_coeffs)            
            for l in range(0, 4):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores[i] = z_scores[i] + score
                #if math.isinf(z_scores[i]):
                    #print(f'z-score is inf: prob = {prob}, z-score {st.norm.ppf(1-(1-prob)/2)}')
            
        #if (contains_match is not True):
            #print(f'Match not found for index {idx} and {n_nearest_neighbors} neighbors')
            #continue
        
        n_matches = n_matches + 1
        max_index, max_z_score = max(enumerate(z_scores), key=operator.itemgetter(1))
        matching_index = nn_indices[max_index]
        dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            loc_count = loc_count + 1;            
        else:
            #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_z_score}.')            
            matching_index = nn_indices[true_match_idx]
            dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
            #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')
                
    loc_precision = (loc_count*1.0) / n_matches    
    #print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
    print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/Voting', loc_precision, n_nearest_neighbors)

## Place Voting using Global Spectral Analysis


In [ ]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
start = time.time()
tree = spatial.KDTree(desc_positives)
end = time.time()
print(f'Duration for building the kd-tree {(end - start)}s')   
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds_test.anchor_poses
anchor_clouds = ds_test.anchors
anchor_features = test_set.anchor_features

positive_poses = ds_test.positive_poses
positive_clouds = ds_test.positives
positive_features = test_set.anchor_features

tapers, eigenvalues, taper_order = spectralanalysis.SHReturnTapers(10.3, 7)
n_bands = 7
for n_nearest_neighbors in tqdm(range(2,21)):        
    n_matches = 0    
    loc_count = 0    
    final_count = 0
    dur_neighbor_processing_s = 0
    dur_s2_s = 0
    dur_spectrum_s = 0
    for idx in range(0, n_test_set):        
    #for idx in range(0, 100):        
        start = time.time()
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)                
        end = time.time()
        dur_neighbor_processing_s = dur_neighbor_processing_s + (end - start)
        
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        z_scores_fused = [0] * n_nearest_neighbors
        z_scores_range = [0] * n_nearest_neighbors
        z_scores_intensity = [0] * n_nearest_neighbors
        z_scores_image = [0] * n_nearest_neighbors                     
        n_true_matches = 0   
        contains_match = False        
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_test_set):
                print(f'ERROR: index {nn_i} is outside of {n_data}')
                break;
                
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True   
                n_true_matches = n_true_matches + 1       
                #print(f'True index = {i}')
                
            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
            a_intensity = anchor_features[idx][1,:,:]
            p_intensity = positive_features[nn_i][1,:,:]
            a_img = anchor_features[idx][2,:,:]
            p_img = positive_features[nn_i][2,:,:]
                                                 
            start_s2 = time.time()
            a_range_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_range_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)
            
            a_intensity_coeffs = pyshtools.expand.SHExpandDH(a_intensity, sampling=1)
            p_intensity_coeffs = pyshtools.expand.SHExpandDH(p_intensity, sampling=1)
            
            a_img_coeffs = pyshtools.expand.SHExpandDH(a_img, sampling=1)
            p_img_coeffs = pyshtools.expand.SHExpandDH(p_img, sampling=1)
            end_s2 = time.time()
            dur_s2_s = dur_s2_s + (end_s2 - start_s2)
                        
                
            start_spectrum = time.time()
            saa_range = spectralanalysis.spectrum(a_range_coeffs)            
            saa_intensity = spectralanalysis.spectrum(a_intensity_coeffs)    
            saa_img = spectralanalysis.spectrum(a_img_coeffs)    
            saa = np.empty([n_features, saa_range.shape[0]])
            saa[0,:] = saa_range
            saa[1,:] = saa_intensity
            saa[2,:] = saa_img
            #saa = np.mean(saa, axis=0)
            saa = np.amax(saa, axis=0)
            
            spp_range = spectralanalysis.spectrum(p_range_coeffs)            
            spp_intensity = spectralanalysis.spectrum(p_intensity_coeffs)    
            spp_img = spectralanalysis.spectrum(p_img_coeffs)    
            spp = np.empty([n_features, spp_range.shape[0]])
            spp[0,:] = spp_range
            spp[1,:] = spp_intensity
            spp[2,:] = spp_img
            #spp = np.mean(spp, axis=0)
            spp = np.amax(spp, axis=0)
            
            sap_range = spectralanalysis.cross_spectrum(a_range_coeffs, p_range_coeffs)            
            sap_intensity = spectralanalysis.cross_spectrum(a_intensity_coeffs, p_intensity_coeffs)    
            sap_img = spectralanalysis.cross_spectrum(a_img_coeffs, p_img_coeffs)    
            sap = np.empty([n_features, sap_range.shape[0]])
            sap[0,:] = sap_range
            sap[1,:] = sap_intensity
            sap[2,:] = sap_img
            #sap = np.mean(sap, axis=0)
            sap = np.amax(sap, axis=0)
            
            #saa = spectralanalysis.spectrum(a_coeffs)
            #spp = spectralanalysis.spectrum(p_coeffs)
            #sap = spectralanalysis.cross_spectrum(a_coeffs, p_coeffs)
            
            #admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)                                    
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap, saa, spp, tapers)
            end_spectrum = time.time()
            dur_spectrum_s = dur_spectrum_s + (end_spectrum - start_spectrum)
            
            
            for l in range(0, 10):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_fused[i] = z_scores_fused[i] + score  
            
                             

            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_range, saa_range, spp_range, tapers)                        
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_range[i] = z_scores_range[i] + score
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_intensity, saa_intensity, spp_intensity, tapers)                            
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_intensity[i] = z_scores_intensity[i] + score                           
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)                            
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_image[i] = z_scores_image[i] + score                                                                   
            
            
        if (contains_match is not True):            
            continue
        
                
        #print(f'z_score > 2 = {np.sum(np.array(z_scores_range) > 3.8)} range, {np.sum(np.array(z_scores_intensity) > 20)} intensity')
        #print(f'true matches: {n_true_matches}')
                
        
        # normalize values
        z_scores_fused = np.array(z_scores_fused) / (n_bands)
        z_scores_range = np.array(z_scores_range) / (n_bands)
        z_scores_intensity = np.array(z_scores_intensity) / (n_bands)
        z_scores_image = np.array(z_scores_image) / (n_bands)
        
#        print(z_scores_range)
 #       print(z_scores_intensity)
  #      print(z_scores_image)
        
        
        max_index_fused, max_z_score_fused = max(enumerate(z_scores_fused), key=operator.itemgetter(1))
        max_index_range, max_z_score_range = max(enumerate(z_scores_range), key=operator.itemgetter(1))
        max_index_intensity, max_z_score_intensity = max(enumerate(z_scores_intensity), key=operator.itemgetter(1))        
        max_index_image, max_z_score_image = max(enumerate(z_scores_image), key=operator.itemgetter(1))
        
        #score_mean = np.mean([z_scores_range,z_scores_intensity,z_scores_image], axis=0)
        #print(f'score_mean: {score_mean}')
        #conf_mask = np.logical_or(np.logical_or(z_scores_range > 3.5, z_scores_intensity > 3.5), z_scores_image > 3.5)
        #print(f'conf_mask: {conf_mask}')
        #if len(score_mean[conf_mask])==0:
#            continue
#        idx_map = np.arange(0,len(score_mean))
        #print(f'idx: {idx}')
        
#        score_std = np.std([z_scores_range,z_scores_intensity,z_scores_image], axis=0)
#        score_std = score_std[conf_mask]
        #print(f'score_std: {score_std}')
#        min_index_std, min_z_score_std = min(enumerate(score_std), key=operator.itemgetter(1))
#        max_index = idx_map[min_index_std]
        #print(f'max_index: {max_index}')
        
        
        n_matches = n_matches + 1
     #   max_scores = np.add(np.add(z_scores_range, z_scores_intensity), z_scores_image)
      #  max_index, max_z_score = max(enumerate(max_scores), key=operator.itemgetter(1))
        
        #print(f'max range: {max_z_score_range}, max intensity: {max_z_score_intensity}')
        #max_index = max_index_range if max_z_score_range > max_z_score_intensity else max_index_intensity        
        #max_score = max_z_score_range if max_z_score_range > max_z_score_intensity else max_z_score_intensity
        #max_index = max_index_range
        #max_score = max_z_score_range
        
        #max_index = max_index if max_score > max_z_score_image else max_index_image
        
        
        #max_index = max_index_range if max_z_score_range > max_z_score_image else max_index_image
        
        max_index = max_index_fused
        matching_index = nn_indices[max_index]        
        dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            loc_count = loc_count + 1;            
            #print('successful')
        #else:
            #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_score}.')            
            #matching_index = nn_indices[true_match_idx]
            #dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
            #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')
                
    loc_recall = (loc_count*1.0) / n_matches    
    loc_precision = (loc_count*1.0) / n_matches    
    #print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
    print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/WindowedVoting', loc_precision, n_nearest_neighbors)
    #print(f'Duration: {dur_neighbor_processing_s/n_test_set}s')    
    #print(f'Duration S^2 Transform: {dur_s2_s/n_test_set}s')
    #print(f'Duration Spectrum: {dur_spectrum_s/n_test_set}s')


In [ ]:
0.9404761904761905
0.9176470588235294
0.8863636363636364
0.8777777777777778
0.8666666666666667

l_max = 15/15
0.9523809523809523
0.9411764705882353
0.9090909090909091
0.8888888888888888
0.8888888888888888

l_max = 10/10
0.9642857142857143
0.9529411764705882
0.9090909090909091
0.8888888888888888
0.8777777777777778
0.8461538461538461
0.8586956521739131
0.8602150537634409
0.8602150537634409
0.8494623655913979
0.8494623655913979

l_max = 7/7
0.9642857142857143
0.9529411764705882
0.9090909090909091
0.8888888888888888
0.8888888888888888
0.8571428571428571
0.8586956521739131
0.8602150537634409
0.8494623655913979

lmax = 7/5
0.9642857142857143
0.9529411764705882
0.9090909090909091
0.8777777777777778
0.8888888888888888
0.8571428571428571
0.8695652173913043
0.8709677419354839
0.8602150537634409
0.8494623655913979
0.8494623655913979
0.8421052631578947
0.8421052631578947
0.8421052631578947
0.8526315789473684

In [ ]:
pytorch_total_params = sum(p.numel() for p in net.parameters())
print(pytorch_total_params)